In [852]:
import numpy as np
import pandas as pd

In [853]:
def cartesian_product(*choices : np.array) -> pd.DataFrame:
    num_axis = len(choices)
    return pd.DataFrame(np.copy(np.vstack(np.vstack(np.meshgrid(*choices))).reshape(num_axis, -1).T))


In [854]:
def _cartesian_success_results(attribute: int, discipline: int, num_dice: int = 2, applicable_focus: bool = False) -> pd.DataFrame:
    d20 = np.arange(1, 21)
    tn = attribute + discipline
    cartesian_df = cartesian_product(*[d20] * num_dice)
    results_df = pd.DataFrame()
    results_df["base_successes"] = (cartesian_df <= tn).sum(axis="columns")
    results_df["critical_successes"] = (cartesian_df == 1).sum(axis="columns")
    results_df["focus_successes"] = (cartesian_df <= discipline ).sum(axis="columns")
    if applicable_focus:
        results_df["successes"] = results_df.loc[:, ["base_successes", "focus_successes"]].sum(axis="columns")
    else:
        results_df["successes"] = results_df.loc[:, ["base_successes", "critical_successes"]].sum(axis="columns")
    return results_df
    

In [855]:
def _create_success_probability_table(enumerated_results: pd.DataFrame, normalize: bool = True) -> pd.DataFrame:
    prob_table_s = enumerated_results["successes"].value_counts(normalize=normalize, sort=False).sort_index(ascending=False).rename("exact successes")
    cum_prob_s = prob_table_s.cumsum().rename("cumulative successes")
    tab_df = pd.concat([prob_table_s, cum_prob_s], axis=1,).fillna(0).astype(prob_table_s.dtype)
    return tab_df

In [856]:
def success_prob_table(attribute: int, discipline: int, num_dice: int = 2, applicable_focus: bool = False, normalize: bool = True) -> pd.DataFrame:
    cr_df = _cartesian_success_results(attribute=attribute, discipline=discipline, num_dice=num_dice, applicable_focus=applicable_focus)
    p_table = _create_success_probability_table(cr_df, normalize=normalize)
    return p_table

In [857]:
def generate_success_frequency_map(
    attribute_min: int = 7,
    attribute_max: int = 12,
    discipline_min: int = 0,
    discipline_max: int = 5,
    dice_min: int = 1,
    dice_max: int = 5
    ):
    from itertools import product, chain

    attribute_range = range(attribute_min, attribute_max+1)
    discipline_range = range(discipline_min, discipline_max+1)
    target_range = range(attribute_min + discipline_min, attribute_max + discipline_max + 1)
    dice_range = range(dice_min, dice_max+1)

    no_focus_index = product(dice_range, [False], target_range, [0])
    focus_index = product(dice_range, [True], attribute_range, discipline_range)
    index_entries = chain(no_focus_index, focus_index)

    frequency_table_map = dict()
    for index_entry in index_entries:
        num_die, focus, attribute, discipline = index_entry
        freq_table = success_prob_table(attribute, discipline, num_die, focus, False)
        freq_table = freq_table["exact successes"].sort_index().to_list()
        if focus:
            frequency_table_map.setdefault(num_die, dict()).setdefault(focus, dict()).setdefault(attribute, dict())[discipline] = freq_table
        else:
            frequency_table_map.setdefault(num_die, dict()).setdefault(focus, dict())[attribute] = freq_table
    return frequency_table_map



In [858]:
def _cartesian_complications_results(num_dice: int = 2, complications_range_min: int = 1, complications_range_max: int = 5):
    d20 = np.arange(1, 21)
    cartesian_df = cartesian_product(*[d20] * num_dice)
    complications = range(complications_range_min, complications_range_max+1)
    res = pd.DataFrame()
    for complications_range in complications:
        res[f"{complications_range}"] = (cartesian_df > (20-complications_range)).sum(axis="columns")
    res.columns.name="complications_range"
    return res


In [859]:
def _create_complications_probability_table(enumerated_results: pd.DataFrame, normalize: bool = True) -> pd.DataFrame:
   value_series = [enumerated_results[col_name].value_counts(sort=False, normalize=normalize).sort_index() for col_name in enumerated_results]
   res = pd.concat(value_series, axis="columns").fillna(0).astype(value_series[0].dtype)
   return res

In [860]:
def complications_probability_table(num_dice: int = 2, complications_range_min: int = 1, complications_range_max: int = 5, normalize=True):
    ccr_df = _cartesian_complications_results(num_dice=num_dice, complications_range_min=complications_range_min, complications_range_max=complications_range_max)
    p_table = _create_complications_probability_table(ccr_df, normalize=normalize)
    return p_table

In [861]:
def generate_complications_frequency_map(
    complications_range_min: int = 1,
    complications_range_max: int = 5,
    dice_min: int = 1,
    dice_max: int = 5,
):
    res = dict()
    for dice in range(dice_min, dice_max+1):
        for column_name in (f_table:=complications_probability_table(num_dice=dice, complications_range_min=complications_range_min, complications_range_max=complications_range_max, normalize=False)):
            res.setdefault(dice, dict())[column_name] = f_table[column_name].sort_index().to_list()
    return res


    

In [862]:
def generate_static_files():
    import json

    json.dump(generate_success_frequency_map(), open("../sta-dice-roller/static-data/successes.json", "w"))
    json.dump(generate_complications_frequency_map(), open("../sta-dice-roller/static-data/complications.json", "w"))

In [863]:
def pool_with_assists(lead_dice: list[list[int]], assist_dice: list[list[int]]) -> pd.DataFrame:
    lead_array = np.array(lead_dice).T
    assist_array = np.array(assist_dice).T
    d20= np.arange(1,21)
    cp_df = cartesian_product(*[d20]*(len(lead_dice) + len(assist_dice)))
    successes_df = pd.DataFrame(cp_df.iloc[:, :len(lead_array)])
    successes_df["lead_successes"] = (cp_df.iloc[:, :len(lead_array[1])] <= lead_array[1]).sum(axis="columns")
    successes_df["lead_bonuses"] = (cp_df.iloc[:, :len(lead_array[0])] <= lead_array[0]).sum(axis="columns")
    successes_df["lead_total"] = (successes_df["lead_successes"] + successes_df["lead_bonuses"])
    successes_df["assist_successes"] = (cp_df.iloc[:, len(lead_array[1]):(len(lead_array[1])+len(assist_array[1]))] <= assist_array[1]).sum(axis="columns")
    successes_df["assist_bonuses"] = (cp_df.iloc[:, len(lead_array[0]):(len(lead_array[0])+len(assist_array[1]))] <= assist_array[0]).sum(axis="columns")
    successes_df["assist_total"] = successes_df["assist_successes"] + successes_df["assist_bonuses"]
    successes_df["assist_total"][successes_df["lead_total"] == 0] = 0
    successes_df["grand_total"] = successes_df["lead_total"] + successes_df["assist_total"]
    return successes_df




In [864]:
test_dice = [[1, 8], [1, 8]]
cartesian_results = pool_with_assists([[5,17],[5,17]], [[0,0], [0,0]])
cartesian_results["grand_total"].value_counts(sort=False).sort_index()

0     3600
1    28800
2    69600
3    48000
4    10000
Name: grand_total, dtype: int64